# 持久化 RAG 系統的完整實戰教學

本文是一份工程導向的完整教學稿，目標不是只「跑得動」，而是讓你真正理解：

為什麼這樣設計索引

>- 為什麼 persistence 會出錯
>- 為什麼 Retrieval 策略需要隨資料規模演進
>- 本文將逐步建構一個 可持久化（Persistent）、可擴展（Scalable）、避免常見陷阱 的 RAG 系統。

## 0️⃣ 前言：我們要解決什麼問題？

在實務上，RAG 系統常見三個痛點：

>- 索引與 Embedding 維度不一致，導致查詢時崩潰
>- 向量資料明明存在磁碟，卻怎麼查都查不到
>- 文件一多，單一相似度搜尋開始失準

![caption](Gemini_Generated_Image_pzkcrgpzkcrgpzkc.png)

這三個問題，其實都來自同一件事： RAG 系統是一個「狀態型系統（stateful system）」，而不是一次性腳本。

本文將用 LlamaIndex + FAISS/Qdrant，從資料蒐集一路走到 Advanced Retrieval，並在每一個關鍵點說清楚「為什麼」。

## 1️⃣ 資料準備與文件建模（Document Layer）
為什麼這一步重要？

RAG 系統的品質，80% 來自於 資料建模是否合理。如果一開始文件與 metadata 設計錯誤，後面的 embedding、retrieval 都只是在放大錯誤。

### 資料來源選擇

我們使用 Wikipedia（中文）作為資料來源，原因是：

>- 內容長度中等（適合 chunking 示範）
>- 結構鬆散（接近真實世界資料）
>- 主題明確（方便驗證 retrieval 正確性）

### Metadata 設計

每份文件都會附加 metadata，例如：

- author：漫畫作者

為什麼要這樣做？

Metadata 並不是裝飾品，而是：

>- Retrieval 階段的「搜尋空間約束條件」
>- 未來進行 metadata filtering / auto-retrieval 的基礎

In [ ]:
from llama_index.readers.file import FlatReader
from llama_index.core import SimpleDirectoryReader#, SummaryIndex

In [ ]:
wiki_titles = ["鋼之鍊金術師", "一拳超人", "ONE_PIECE", "東京喰種"]
wiki_metadatas = {
    "鋼之鍊金術師": {
        "author": "荒川弘",
    },
    "一拳超人": {
        "author": "ONE",
    },
    "ONE_PIECE": {
        "author": "尾田榮一郎",
    },
    "東京喰種": {
        "author": "石田翠",
    },
}

In [ ]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='zh-tw')

In [ ]:
from pathlib import Path

for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [ ]:
from glob import glob

docs = SimpleDirectoryReader(input_files=[f for f in glob("data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

In [ ]:
docs[0].metadata

In [ ]:
import os

os.chdir("../")

from initialization import credential_init

credential_init()

建立nodes

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=128,
    chunk_overlap=0,
    separators=["\n\n", "\n", "。", "，", " "]
).split_text

semantic_splitter_node_parser = SemanticSplitterNodeParser.from_defaults(
    embed_model = embed_model,
    sentence_splitter=text_splitter,
    include_metadata=True,
    include_prev_next_rel=True,
    breakpoint_percentile_threshold=60,
)

raw_nodes = semantic_splitter_node_parser.get_nodes_from_documents(docs)

## FAISS + VectorStoreIndex Persist

- 建立 embedding
- 建立 vectorstore
- 建立 storage context
- 建立 vectorstoreindex

In [ ]:
import faiss
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.faiss import FaissVectorStore

persist_dir = "./week_2/storage_faiss"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
)

In [ ]:
from src.ollama_connection import llama_index_ollama

ollama_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

In [ ]:
faiss_index = VectorStoreIndex(
    raw_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
)

In [ ]:
faiss_query_engine = faiss_index.as_query_engine(similarity_top_k=5,
                                                 response_mode="compact",
                                                 llm=ollama_llm)

In [ ]:
output = faiss_query_engine.query("誰是最強的光頭?")

In [ ]:
output.response

儲存 Storage

In [ ]:
storage_context.persist(persist_dir="./week_2/storage_faiss")

讀取 Storage 重建 Index

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# 【步驟 A】從本地路徑還原向量存儲物件
# 注意：不要手動建立空的 faiss.IndexFlatL2，直接從 persist_dir 讀取
vector_store = FaissVectorStore.from_persist_dir("./week_2/storage_faiss")

# 【步驟 B】構建存儲上下文
storage_context_loaded = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir="./week_2/storage_faiss"
)

# 【步驟 C】載入索引並建立查詢引擎
faiss_vector_index_loaded = load_index_from_storage(
    storage_context_loaded,
    embed_model=embed_model
)

faiss_query_engine_loaded = faiss_vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    embed_model=embed_model,
    llm=ollama_llm
)

In [ ]:
output = faiss_query_engine.query("誰是最強的光頭?")

In [ ]:
output.response

### 更新 Storage Context

In [ ]:
title = "银之匙_Silver_Spoon"

page = wiki_wiki.page(title)
wiki_text = page.text

In [ ]:
from llama_index.core import Document

new_doc = Document(text=wiki_text, metadata={"author": "荒川弘"})

new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
# 確保 nodes 已經被向量化
for node in new_nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
# new_nodes[0]

更新 vectorstore index

In [ ]:
faiss_vector_index_loaded.insert_nodes(new_nodes)

In [ ]:
faiss_query_engine_updated = faiss_vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    embed_model=embed_model,
    llm=ollama_llm
)

In [ ]:
output = faiss_query_engine_updated.query("銀之匙的創作背景")

In [ ]:
output.response

In [ ]:
faiss_vector_index_loaded.storage_context.persist(persist_dir="./week_2/storage_faiss_new")

實驗是否可以正常執行

In [ ]:
vector_store = FaissVectorStore.from_persist_dir("./week_2/storage_faiss_new")

storage_context_loaded = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir="./week_2/storage_faiss_new"
)

faiss_vector_index_loaded = load_index_from_storage(
    storage_context_loaded,
    embed_model=embed_model
)

faiss_query_engine_loaded = faiss_vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    embed_model=embed_model,
    llm=ollama_llm
)

In [ ]:
output = faiss_query_engine_loaded.query("銀之匙的創作背景")

In [ ]:
output.response

### Shared Storage Context 與 Multi-Index 管理（進階）

每次呼叫 VectorStoreIndex.from_documents，LlamaIndex 都會：

- 先建立一個臨時 UUID index
- 再讓你手動指定 index_id

如果沒有清理，就會：

- 多存一個「沒人用的 index」

解法核心觀念

- StorageContext 是 索引一致性的邊界
- index_id 必須同步更新到 index_store

In [ ]:
import shutil

persist_dir = "./week_2/storage_shared"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
)

# add documents to index
docs = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

    nodes = semantic_splitter_node_parser.get_nodes_from_documents([doc])
    
    index = VectorStoreIndex(
        nodes,
        storage_context=storage_context,
        embed_model=embed_model,
    )
    # 2. To strictly clean up the auto-generated one:
    old_id = index.index_id
    print(old_id)
    # Manually set the index_id on the index_struct
    index.set_index_id(comic_name)
    # index.index_struct.index_id = wiki_title
    
    # Sync the index_id back to the storage context's index store
    storage_context.index_store.add_index_struct(index.index_struct)

    # FIX: Use a try-except or check against the dictionary keys correctly
    # Usually, storage_context.index_store.index_structs is the internal dict
    try:
        storage_context.index_store.delete_index_struct(old_id)
    except Exception:
        # If it's already gone or doesn't exist, just move on
        pass

# Persist ONCE after the loop (more efficient than inside the loop)
storage_context.persist(persist_dir=persist_dir)

讀取指定index_id

In [ ]:
vector_store_loaded = FaissVectorStore.from_persist_dir("./week_2/storage_shared")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store_loaded,
    persist_dir="./week_2/storage_shared",
)

In [ ]:
vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=2,
    llm=ollama_llm,
)

In [ ]:
# query_engine_tokyo_ghoul.query("解釋赫子")

### 加入新的index

In [ ]:
# storage_context.index_store.index_structs()

In [ ]:
from llama_index.core import VectorStoreIndex

# 建立新的index
new_index = VectorStoreIndex(
    nodes=new_nodes,               # 你要加入的新資料
    storage_context=storage_context,
    embed_model=embed_model,
)

old_id = new_index.index_id
new_index.set_index_id("銀之匙")

# Sync the index_id back to the storage context's index store
storage_context.index_store.add_index_struct(index.index_struct)

storage_context.index_store.delete_index_struct(old_id)

In [ ]:
storage_context.persist(persist_dir="./week_2/storage_shared_new")

### 更新舊的index

In [ ]:
title = "石田翠"

page = wiki_wiki.page(title)
wiki_text = page.text

new_doc = Document(text=wiki_text)

tokyo_ghoul_new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
# 確保 nodes 已經被向量化
for node in tokyo_ghoul_new_nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

In [ ]:
vector_index_tokyo_ghoul.insert_nodes(tokyo_ghoul_new_nodes)
storage_context.persist(persist_dir="./week_2/storage_shared_tokyo_ghoul_new")

檢查

In [ ]:
vector_store_loaded = FaissVectorStore.from_persist_dir("./week_2/storage_shared_tokyo_ghoul_new")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store_loaded,
    persist_dir="./week_2/storage_shared_tokyo_ghoul_new",
)

vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=5,
    llm=ollama_llm,
)

In [ ]:
# query_engine_tokyo_ghoul.query("東京喰種作者的人生經歷")

## Qdrant + VectorStoreIndex Persist

In [ ]:
await client.close()

In [ ]:
from qdrant_client import AsyncQdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore

"""
await client.close()
"""

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

persist_dir = "./week_2/storage_qdrant"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

storage_context = StorageContext.from_defaults(
    vector_store=qdrant_vector_store,
)

In [ ]:
qdrant_index = VectorStoreIndex(
    raw_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
    use_async=True
)

In [ ]:
storage_context.persist(persist_dir="./week_2/storage_qdrant")

測試還原 qdrant_index

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_shared_qdrant",
    vector_store=qdrant_vector_store,
)

In [ ]:
# 4. 重建 index
qdrant_vector_index = load_index_from_storage(
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
)

In [ ]:
qdrant_query_engine = qdrant_index.as_query_engine(similarity_top_k=5,
                                                   llm=ollama_llm)

In [ ]:
# await qdrant_query_engine.aquery("海賊王最新劇情")

### 更新Storage Context

In [ ]:
title = "银之匙_Silver_Spoon"

page = wiki_wiki.page(title)
wiki_text = page.text

In [ ]:
new_doc = Document(text=wiki_text, metadata={"author": "荒川弘"})

new_nodes = semantic_splitter_node_parser.get_nodes_from_documents([new_doc])

In [ ]:
# 確保 nodes 已經被向量化
for node in new_nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
# async version
await qdrant_index.ainsert_nodes(new_nodes)

In [ ]:
qdrant_query_engine = qdrant_index.as_query_engine(similarity_top_k=10,
                                                   llm=ollama_llm)

output = await qdrant_query_engine.aquery("銀之匙的創作背景")

In [ ]:
output.response

In [ ]:
storage_context.persist()

測試Storage Context是否被成功persist

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_qdrant",
    vector_store=qdrant_vector_store,
)

qdrant_vector_index = load_index_from_storage(
    storage_context=storage_context,
    embed_model=embed_model,
    llm=ollama_llm,
)

qdrant_query_engine = qdrant_vector_index.as_query_engine(similarity_top_k=10,
                                                   llm=ollama_llm)

output = await qdrant_query_engine.aquery("銀之匙的創作背景")

In [ ]:
output.response

### Shared Storage Context 與 Multi-Index 管理（進階）

In [ ]:
persist_dir = "./week_2/storage_qdrant_shared"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_shared")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    vector_store=qdrant_vector_store,
)

# add documents to index
docs = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

for doc in docs:
    comic_name = Path(doc.metadata['file_name']).stem
    doc.metadata.update(wiki_metadatas[comic_name])

    nodes = semantic_splitter_node_parser.get_nodes_from_documents([doc])
    
    index = VectorStoreIndex(
        nodes,
        storage_context=storage_context,
        embed_model=embed_model,
        use_async=True
    )

    old_id = index.index_id
    print(old_id)
    index.set_index_id(comic_name)

    storage_context.index_store.add_index_struct(index.index_struct)

    try:
        storage_context.index_store.delete_index_struct(old_id)
    except Exception:
        pass

# Persist ONCE after the loop (more efficient than inside the loop)
storage_context.persist(persist_dir=persist_dir)

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_shared")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_qdrant_shared",
    vector_store=qdrant_vector_store,
)

vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "東京喰種",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=10,
    llm=ollama_llm,
)

output = await query_engine_tokyo_ghoul.aquery("東京喰種作者的人生經歷")

In [ ]:
output.response

#### 加入新的index

In [ ]:
# 建立新的index
new_index = VectorStoreIndex(
    nodes=new_nodes,               # 你要加入的新資料
    storage_context=storage_context,
    embed_model=embed_model,
    use_async=True
)

old_id = new_index.index_id
new_index.set_index_id("銀之匙")

# Sync the index_id back to the storage context's index store
storage_context.index_store.add_index_struct(new_index.index_struct)

storage_context.index_store.delete_index_struct(old_id)

storage_context.persist(persist_dir="./week_2/storage_qdrant_shared_new")

In [ ]:
await aclient.close()

aclient = AsyncQdrantClient(path="week_2/langchain_qdrant_shared")

qdrant_vector_store = QdrantVectorStore(
    aclient=aclient,
    collection_name="my_document" # This collection will hold your vectors and associated documents.
)

storage_context = StorageContext.from_defaults(
    persist_dir="./week_2/storage_qdrant_shared_new",
    vector_store=qdrant_vector_store,
)

vector_index_tokyo_ghoul = load_index_from_storage(
    storage_context,
    index_id = "銀之匙",
    embed_model=embed_model
)

query_engine_tokyo_ghoul = vector_index_tokyo_ghoul.as_query_engine(
    similarity_top_k=10,
    llm=ollama_llm,
)

output = await query_engine_tokyo_ghoul.aquery("銀之匙的創作背景")

In [ ]:
output.response

#### 更新Index

就留作練習吧~~

# Advanced Node Parser - Part 1

## 2️⃣ Chunking 與 Embedding：建立語意空間
### 2.1 Chunking 策略（SentenceSplitter）

我們使用 SentenceSplitter(chunk_size=256)。

為什麼不是越大越好？

>- Chunk 太大 → embedding 混合多個主題，相似度失準
>- Chunk 太小 → 語意破碎，LLM 回答品質下降

256 tokens 是在「語意完整性」與「檢索精度」之間的折衷。

### 2.2 Embedding Model 的角色（⚠️核心觀念）

Embedding Model 不是可隨意更換的零件。

它實際上定義了：

>- 向量的維度
>- 向量空間的幾何結構

👉 索引與查詢必須使用同一個 embedding model，否則必然出錯。

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager
from llama_index.core.node_parser import SentenceSplitter


llm = OpenAI("gpt-4o-mini")
callback_manager = CallbackManager([LlamaDebugHandler()])
splitter = SentenceSplitter(chunk_size=256)

## 3️⃣ Vector Store + Metadata Auto-Retrieval
### 3.1 為什麼需要 Metadata Filters？

當資料規模變大時：

- 單純靠相似度搜尋 → 雜訊急遽增加
- Metadata 可以先縮小搜尋子空間

### 3.2 Auto-Retrieval 的工作流程

VectorIndexAutoRetriever 的核心思想是：

User Query
→ LLM 推斷可能的 metadata 條件
→ 套用 filters 進行向量搜尋
→ 回傳更精準的節點

這讓 Retrieval 從「被動比對」進化成「語意驅動的檢索策略」。

In [ ]:
import faiss
from llama_index.core import StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
storage_context.docstore.docs

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[splitter],
    callback_manager=callback_manager,
    embed_model=embed_model
)

# add documents to index
for wiki_title in wiki_titles:
    index.insert(docs_dict[wiki_title])

In [ ]:
len(storage_context.docstore.docs)

In [ ]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info="brief biography of celebrities",
    metadata_info=[
        MetadataInfo(
            name="author",
            type="str",
            description=(
                "漫畫作者"
            ),
        ),
        MetadataInfo(
            name="country",
            type="str",
            description=(
                "Country of the celebrity, one of [United States, Barbados,"
                " Portugal]"
            ),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index,
    vector_store_info=vector_store_info,
    llm=llm,
    callback_manager=callback_manager,
    max_top_k=10000,
)

如果 metadata 設計錯誤：
- Auto-retrieval 無法運作
- Filter 只能硬編碼
- Retrieval scalability 直接封頂

In [ ]:
nodes = retriever.retrieve(
    "最強的光頭是誰?"
)

In [ ]:
nodes

## 6️⃣ Shared Storage Context 與 Multi-Index 管理（進階）
為什麼會出現「幽靈索引」？

每次呼叫 VectorStoreIndex.from_documents，LlamaIndex 都會：

- 先建立一個臨時 UUID index
- 再讓你手動指定 index_id

如果沒有清理，就會：

- 多存一個「沒人用的 index」

解法核心觀念

- StorageContext 是 索引一致性的邊界
- index_id 必須同步更新到 index_store

In [ ]:
import shutil

persist_dir = "./week_2/storage_shared"

# Overwrite logic: remove the folder if it exists
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
)

# add documents to index
for wiki_title in wiki_titles:

    index = VectorStoreIndex.from_documents(
        [docs_dict[wiki_title]],
        storage_context=storage_context,
        transformations=[splitter],
        embed_model=embed_model,
    )
    # 2. To strictly clean up the auto-generated one:
    old_id = index.index_id
    print(old_id)
    # Manually set the index_id on the index_struct
    index.set_index_id(wiki_title)
    # index.index_struct.index_id = wiki_title
    
    # Sync the index_id back to the storage context's index store
    storage_context.index_store.add_index_struct(index.index_struct)

    # FIX: Use a try-except or check against the dictionary keys correctly
    # Usually, storage_context.index_store.index_structs is the internal dict
    try:
        storage_context.index_store.delete_index_struct(old_id)
    except Exception:
        # If it's already gone or doesn't exist, just move on
        pass

# Persist ONCE after the loop (more efficient than inside the loop)
storage_context.persist(persist_dir=persist_dir)

**讀取 Multi-Index storage_context**

In [ ]:
vector_store_loaded = FaissVectorStore.from_persist_dir("./week_2/storage_shared")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store_loaded,
    persist_dir="./week_2/storage_shared",
)

In [ ]:
vector_index_loaded = load_index_from_storage(
    storage_context,
    index_id = wiki_title
)

In [ ]:
query_engine = vector_index_loaded.as_query_engine(
    similarity_top_k=2,
    llm=llm,
    embed_model=embed_model,
    # the target key defaults to `window` to match the node_parser's default
)

In [ ]:
query_engine.query("解釋赫子")

## 7️⃣ Advanced Retrieval Strategies
### 7.1 Small-to-Big Retrieval（Sentence Window）

適合情境：

- 長文件
- 查詢精細事實

作法：

- Index 用單句 embedding
- 回傳時補上上下文 window

### VectorStoreIndex with SentenceWindowNodeParser

In [ ]:
from llama_index.core import Document

# If docs_dict contains strings, wrap them first:
documents = [docs_dict[wiki_title] for wiki_title in wiki_titles]

In [ ]:
a = SentenceSplitter.from_defaults()

In [ ]:
len(a.get_nodes_from_documents(documents[:1]))

In [ ]:
a.get_nodes_from_documents(documents[:1])[0]

In [ ]:
type(documents[0])

In [ ]:
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

sentence_splitter = SentenceSplitter.from_defaults()

# create the sentence window node parser w/ default settings
# applying sentence_splitter in SentenceWindowNodeParser: https://github.com/run-llama/llama_index/issues/14029
sw_parser = SentenceWindowNodeParser.from_defaults(
    window_size=2,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    sentence_splitter=sentence_splitter.split_text
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

# create the pipeline with transformations
sw_pipeline = IngestionPipeline(
    transformations=[
        sw_parser,
    ],
)

sw_nodes = sw_pipeline.run(documents=documents)

faiss_index_loaded = faiss.IndexFlatL2(d)
sw_vector_store = FaissVectorStore(faiss_index=faiss_index_loaded)

sw_storage_context = StorageContext.from_defaults(vector_store=sw_vector_store)

In [ ]:
len(sw_nodes)

In [ ]:
sw_index = VectorStoreIndex(
    sw_nodes,
    storage_context=sw_storage_context,
    embed_model=embed_model
)

In [ ]:
len(sw_index.index_struct.nodes_dict)

### VectorStoreIndex with SentenceSplitter

In [ ]:
# base node parser is a sentence splitter
base_parser = SentenceSplitter()

base_nodes = base_parser.get_nodes_from_documents(documents)

faiss_index_loaded = faiss.IndexFlatL2(d)
base_vector_store = FaissVectorStore(faiss_index=faiss_index_loaded)

base_storage_context = StorageContext.from_defaults(
    vector_store=base_vector_store,
)

base_index = VectorStoreIndex(base_nodes,
                              storage_context=base_storage_context,
                              embed_model=embed_model)

In [ ]:
len(base_nodes)

### Query

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

sw_query_engine = sw_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = sw_query_engine.query(
    "最強的光頭是誰?"
)
print(window_response)

In [ ]:
base_query_engine = base_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
)
base_response = base_query_engine.query(
    "最強的光頭是誰?"
)
print(base_response)

## 7.2 RecursiveRetriever 與 IndexNode

隨著以 LLM 為核心的系統從小型、扁平的文件集合，擴展到更複雜的知識庫，單純依賴相似度搜尋的檢索方式已逐漸不足以應付實際需求。真實世界的資料往往具有 **階層性（hierarchical）**、**模組化（modular）** 與 **多層次粒度（multi-granular）** 的特性：書籍由章節與段落組成、知識庫依領域劃分，而摘要自然位於詳細內容之上。  
LlamaIndex 正是透過 **RecursiveRetriever** 與 **IndexNode** 來解決這類問題。

### IndexNode

**IndexNode** 是一種特殊的節點（node），它不僅包含文字內容，還 **指向另一個索引（index）或檢索器（retriever）**。從概念上來看，它是一座連接不同抽象層級的「橋樑」。

你可以將 IndexNode 想成：
- 一段摘要、標籤或描述文字（例如：「第 2 章：檢索方法」）
- 再加上一個指向更深層索引的參考

這使得節點可以代表 **一整組節點的集合**，而不只是單一的文字片段。與其將所有低層級的內容一次性嵌入到一個龐大的索引中，不如先嵌入高層摘要，並由它們連結到更細節的索引。

### RecursiveRetriever

**RecursiveRetriever** 是一種可以同時處理一般節點與 IndexNode 的檢索器。當它檢索到一個 IndexNode 時，流程並不會就此停止，而是會 **遞迴地呼叫該 IndexNode 所指向的檢索器**，並在下一層繼續進行檢索。

在高層次上，整個檢索流程如下：

1. 先檢索高層節點（通常是摘要或分類）
2. 在結果中辨識出 IndexNode
3. 依照 IndexNode 的指向，進入更深層的檢索器或索引
4. 檢索更細粒度的節點
5. 重複上述流程，直到抵達最底層的節點

這樣的設計讓檢索不再只是單次的平面搜尋，而是 **多階段、具階層性的檢索過程**。

### 為什麼這很重要

將 RecursiveRetriever 與 IndexNode 搭配使用，可以讓你：
- 在不一次嵌入所有資料的情況下，擴展到大型語料庫
- 保留文件原有的結構與階層關係
- 實現「由粗到細」的檢索流程（由整體概覽逐步深入細節）
- 僅在相關的子空間中搜尋，降低雜訊
- 組合不同類型的索引（例如：SummaryIndex → VectorStoreIndex）

### 心智模型（Mental model）

- **IndexNode**：「這段文字代表一個集合，可以從這裡往下深入。」
- **RecursiveRetriever**：「先檢索，再順著連結一路往下，直到找到合適的細節層級。」

兩者結合，構成了 LlamaIndex 中 **結構化、階層式檢索管線** 的核心基礎，使你能打造更精準且高效率的 RAG 系統。


In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode

In [ ]:
# define top-level nodes and vector retrievers
nodes = []
vector_query_engines = {}
vector_retrievers = {}

for wiki_title in wiki_titles:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        [docs_dict[wiki_title]],
        transformations=[splitter],
        callback_manager=callback_manager,
        embed_model=embed_model,
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    vector_query_engines[wiki_title] = vector_query_engine
    vector_retrievers[wiki_title] = vector_index.as_retriever()

    # save summaries
    out_path = Path("week_2/summaries") / f"{wiki_title}.txt"
    if not out_path.exists():
        # use LLM-generated summary
        summary_index = SummaryIndex.from_documents(
            [docs_dict[wiki_title]], callback_manager=callback_manager
        )

        summarizer = summary_index.as_query_engine(
            response_mode="tree_summarize", llm=llm
        )
        response = await summarizer.aquery(
            f"請給我{wiki_title}的總結"
        )

        wiki_summary = response.response
        Path("week_2/summaries").mkdir(exist_ok=True)
        with open(out_path, "w", encoding="utf-8") as fp:
            fp.write(wiki_summary)
    else:
        with open(out_path, "r", encoding="utf-8") as fp:
            wiki_summary = fp.read()

    print(f"**Summary for {wiki_title}: {wiki_summary}")
    node = IndexNode(text=wiki_summary, index_id=wiki_title)
    nodes.append(node)

In [ ]:
nodes

In [ ]:
# define top-level retriever
splitter = SentenceSplitter(chunk_size=64, chunk_overlap=0)

top_vector_index = VectorStoreIndex(
    nodes, transformations=[splitter], callback_manager=callback_manager, embed_model=embed_model
)
top_vector_retriever = top_vector_index.as_retriever(similarity_top_k=1)

In [ ]:
len(nodes)

In [ ]:
len(top_vector_index.docstore.docs)

In [ ]:
len(top_vector_index.vector_store.data.embedding_dict)

In [ ]:
top_vector_retriever.retrieve("東京喰種")

In [ ]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": top_vector_retriever, **vector_retrievers},
    # query_engine_dict=vector_query_engines,
    verbose=True,
)

In [ ]:
nodes_example = recursive_retriever.retrieve(
    "東京喰種"
)

## 8️⃣ 總結：設計原則（請記住這些）

- Embedding Model ≠ 可替換零件
- Index ≠ 單一 FAISS 檔案
- StorageContext ≠ 選配
- Retrieval Strategy 應隨資料規模演進